<a href="https://colab.research.google.com/github/Luzifer998/NIDS_LSTM_Project_V1.ipynb/blob/main/NIDS_LSTM_Project_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout
import tensorflow as tf
from google.colab import drive
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# เชื่อมต่อ Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# ***************************************************************
# *** 1. แก้ไขพาธนี้ด้วยไฟล์ KDDTrain+.txt/csv จริงของคุณ ***
# ***************************************************************
TRAIN_FILE_PATH = '/content/drive/MyDrive/dataset/KDDTrain+.txt'

# ชื่อคอลัมน์มาตรฐานสำหรับ KDD Cup 99/NSL-KDD
FEATURE_NAMES = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
    'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'attack_type', 'successful.'
]

try:
    data = pd.read_csv(TRAIN_FILE_PATH, header=None, names=FEATURE_NAMES, sep=',')
    data = data.drop(columns=['successful.'])
    print(f"✅ โหลดไฟล์: {TRAIN_FILE_PATH} สำเร็จแล้ว!")

except FileNotFoundError:
    print(f"❌ ERROR: ไม่พบไฟล์ที่พาธ {TRAIN_FILE_PATH}! โปรดตรวจสอบพาธ")
    raise

# --- A. การจัดการ Label และ Attack Type จริง ---
# 1. เก็บชื่อชนิดการโจมตีที่เป็นข้อความไว้ก่อน
global_attack_types = data['attack_type'].copy()
# 2. แปลงเป็น Binary (0=Normal, 1=Attack)
data['attack_type'] = data['attack_type'].apply(lambda x: 0 if x == 'normal' else 1)
y = data['attack_type'].values
data = data.drop(columns=['attack_type'])


# --- B. One-Hot Encoding สำหรับข้อมูล Text/Categorical ---
categorical_cols = ['protocol_type', 'service', 'flag']
print(f"\n⚙️ กำลังทำ One-Hot Encoding สำหรับคอลัมน์: {categorical_cols}")

data_encoded = pd.get_dummies(data, columns=categorical_cols, drop_first=True)
X = data_encoded.values
print(f"⭐ Feature Shape (X) หลัง Encoding: {X.shape}, Label Shape (y): {y.shape}")


# --- C. การปรับขนาดข้อมูลฟีเจอร์ (Scaling) ---
global_scaler = StandardScaler()
X_scaled = global_scaler.fit_transform(X)


# --- D. แบ่งข้อมูลและ Reshape สำหรับ LSTM ---
# ต้องแบ่ง global_attack_types ด้วย train_test_split เดียวกันกับ X/y
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)
_, attack_type_test, _, _ = train_test_split(
    global_attack_types, y, test_size=0.2, random_state=42, stratify=y
)
print("✅ เตรียม Attack Type สำหรับชุดทดสอบสำเร็จแล้ว!")

timesteps = 1
features = X_train.shape[1]

X_train_final = X_train.reshape(X_train.shape[0], timesteps, features)
X_test_final = X_test.reshape(X_test.shape[0], timesteps, features)

print(f"\n✅ Input Shape สำหรับ LSTM (X_train_final): {X_train_final.shape}")

✅ โหลดไฟล์: /content/drive/MyDrive/dataset/KDDTrain+.txt สำเร็จแล้ว!

⚙️ กำลังทำ One-Hot Encoding สำหรับคอลัมน์: ['protocol_type', 'service', 'flag']
⭐ Feature Shape (X) หลัง Encoding: (125973, 119), Label Shape (y): (125973,)
✅ เตรียม Attack Type สำหรับชุดทดสอบสำเร็จแล้ว!

✅ Input Shape สำหรับ LSTM (X_train_final): (100778, 1, 119)


In [ ]:
from tensorflow.keras.layers import Bidirectional

# ***************************************************************
# ** Cell 3: สร้างและฝึกโมเดล NIDS-AI (LSTM) - ปรับปรุง **
# ***************************************************************
print("\n--- เริ่มการฝึกโมเดล NIDS-AI (Bi-LSTM) ---")

# 1. กำหนดสถาปัตยกรรมโมเดล
input_shape = (X_train_final.shape[1], X_train_final.shape[2])
model = Sequential()

# *** เปลี่ยนเป็น Bidirectional LSTM ***
model.add(Bidirectional(LSTM(64, return_sequences=True), input_shape=input_shape))
model.add(Dropout(0.3)) # เพิ่ม Dropout เล็กน้อย

model.add(Bidirectional(LSTM(32, return_sequences=False)))
model.add(Dropout(0.3)) # เพิ่ม Dropout เล็กน้อย

model.add(Dense(1, activation='sigmoid'))

# 2. Compile โมเดล (เหมือนเดิม)
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)
model.summary()

# 3. เริ่มการฝึกโมเดล
history = model.fit(
    X_train_final,
    y_train,
    epochs=100, # *** เพิ่มจำนวน Epochs เป็น 100 ***
    batch_size=256,
    validation_split=0.2,
    verbose=1
)

print("\n✅ การฝึกโมเดลเสร็จสมบูรณ์!")


--- เริ่มการฝึกโมเดล NIDS-AI (Bi-LSTM) ---


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional) │ (None, 1, 128)         │        94,208 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 1, 128)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_3 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_11 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 135,489 (529.25 KB)

 Trainable params: 135,489 (529.25 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 16s 17ms/step - accuracy: 0.9396 - loss: 0.2563 - val_accuracy: 0.9834 - val_loss: 0.0410
Epoch 2/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step - accuracy: 0.9849 - loss: 0.0340 - val_accuracy: 0.9915 - val_loss: 0.0284
Epoch 3/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - accuracy: 0.9914 - loss: 0.0255 - val_accuracy: 0.9932 - val_loss: 0.0229
Epoch 4/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.9931 - loss: 0.0201 - val_accuracy: 0.9938 - val_loss: 0.0214
Epoch 5/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9934 - loss: 0.0190 - val_accuracy: 0.9930 - val_loss: 0.0214
Epoch 6/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9936 - loss: 0.0178 - val_accuracy: 0.9934 - val_loss: 0.0206
Epoch 7/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.9937 - loss: 0.0172 - val_accuracy: 0.9936 - val_loss: 0.0204
Epoch 8/100
315/315 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.9939 - loss: 0.0163 -

In [ ]:
# ***************************************************************
# ** Cell 4: ประเมินผลและบันทึกโมเดล **
# ***************************************************************

# 1. ประเมินผลโมเดล (ส่วนนี้ยังคงอยู่เพื่อให้ครบถ้วน)
loss, accuracy = model.evaluate(X_test_final, y_test, verbose=0)
print(f"\n⭐ ความแม่นยำในการทดสอบ (Test Accuracy): {accuracy*100:.2f}%")

# 2. บันทึกโมเดล
NEW_MODEL_NAME = 'nids_lstm_final_model.h5'
try:
    model.save(NEW_MODEL_NAME)
    # *** บรรทัดนี้คือส่วนที่แสดงชื่อไฟล์ที่บันทึก ***
    print(f"=======================================================")
    print(f"💾 **โมเดลถูกบันทึกสำเร็จในชื่อ: {NEW_MODEL_NAME}**")
    print(f"=======================================================")
except Exception as e:
    print(f"❌ ข้อผิดพลาดในการบันทึกโมเดล: {e}")

# 3. วิเคราะห์ผลลัพธ์
y_pred_proba = model.predict(X_test_final)
y_pred = (y_pred_proba > 0.5).astype(int)

print("\n--- 4. รายงานผลลัพธ์ ---")
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


⭐ ความแม่นยำในการทดสอบ (Test Accuracy): 99.69%
💾 **โมเดลถูกบันทึกสำเร็จในชื่อ: nids_lstm_final_model.h5**
788/788 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step

--- 4. รายงานผลลัพธ์ ---
Confusion Matrix:
[[13435    34]
 [   43 11683]]

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     13469
           1       1.00      1.00      1.00     11726

    accuracy                           1.00     25195
   macro avg       1.00      1.00      1.00     25195
weighted avg       1.00      1.00      1.00     25195



In [ ]:
# ***************************************************************
# ** Cell 5: ส่วนที่ 2. โหลดโมเดลสำหรับจำลอง **
# ***************************************************************

# 1. ระบุชื่อไฟล์โมเดล
NEW_MODEL_NAME = '/content/drive/MyDrive/dataset/nids_lstm_model.h5'  # <-- เปลี่ยนตามตำแหน่งจริง

print("\n--- 🚀 เริ่มโหลดโมเดลสำหรับจำลอง ---")

try:
    loaded_model = load_model(NEW_MODEL_NAME)
    print(f"✅ โหลดโมเดลสำเร็จ: {NEW_MODEL_NAME}")
except Exception as e:
    print(f"❌ ไม่พบไฟล์โมเดล {NEW_MODEL_NAME} หรือเกิดข้อผิดพลาดในการโหลด: {e}")
    raise

# 2. Compile โมเดลเพื่อให้พร้อมสำหรับ evaluate/predict
try:
    loaded_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    print("✅ โมเดลถูก compile สำเร็จแล้ว\n")
except Exception as e:
    print(f"❌ เกิดข้อผิดพลาดระหว่าง compile โมเดล: {e}")
    raise

# 3. (ทางเลือก) ทดสอบ evaluate เบื้องต้นเพื่อให้ metric ถูก build
# * ข้ามได้ถ้ายังไม่มีข้อมูลทดสอบตอนนี้ *
try:
    dummy_input = np.zeros((1, 1, loaded_model.input_shape[-1]))
    dummy_output = np.array([0])
    loaded_model.evaluate(dummy_input, dummy_output, verbose=0)
    print("🔧 Initial evaluate เพื่อ build metrics สำเร็จ ✅")
except Exception:
    pass

print("\n--- ✅ โมเดลพร้อมสำหรับการจำลองใน Cell ถัดไป ---")



--- 🚀 เริ่มโหลดโมเดลสำหรับจำลอง ---
✅ โหลดโมเดลสำเร็จ: /content/drive/MyDrive/dataset/nids_lstm_model.h5
✅ โมเดลถูก compile สำเร็จแล้ว

🔧 Initial evaluate เพื่อ build metrics สำเร็จ ✅

--- ✅ โมเดลพร้อมสำหรับการจำลองใน Cell ถัดไป ---


In [ ]:
# ***************************************************************
# ** Cell 5: การจำลอง Real-Time Detection (แสดงผลเป็นตาราง) **
# ***************************************************************

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import load_model
from IPython.display import display

# 1. ฟังก์ชัน Real-Time Prediction ที่สร้างตารางผลลัพธ์
def real_time_predict(reshaped_data_batch, true_labels_batch, loaded_model, threshold=0.9):
    """
    ทำการตรวจสอบภัยคุกคามในชุดข้อมูลขนาดเล็กแบบ Real-time และคืนค่าผลลัพธ์เป็น DataFrame
    """

    # 1. ทำนายความน่าจะเป็น
    probabilities = loaded_model.predict(reshaped_data_batch, verbose=0)

    # 2. แปลงเป็นป้ายกำกับ Binary (0 หรือ 1)
    predictions = (probabilities.flatten() > 0.5).astype(int)

    # 3. คำนวณความแม่นยำของชุดนี้
    batch_accuracy = accuracy_score(true_labels_batch, predictions)

    # 4. สร้างตารางผลลัพธ์
    results_df = pd.DataFrame({
        'คำทำนาย': np.where(predictions == 0, 'ปกติ', 'โจมตี'),
        'ป้ายกำกับจริง': np.where(true_labels_batch == 0, 'ปกติ', 'โจมตี'),
        'สถานะ': np.where(predictions == true_labels_batch, 'ถูกต้อง ✅', 'ผิดพลาด ❌'),
        'ความน่าจะเป็น': probabilities.flatten()
    })

    # ตรวจสอบการแจ้งเตือน
    alert_indices = np.where(results_df['ความน่าจะเป็น'] > threshold)[0]

    if len(alert_indices) > 0:
        print(f"\n🚨 ตรวจพบการบุกรุก {len(alert_indices)} แถว (Threshold = {threshold})")
    else:
        print("\n✅ ไม่พบการบุกรุกที่มีความน่าจะเป็นสูงในชุดนี้")

    return results_df, batch_accuracy


# 2. โหลดโมเดล
MODEL_TO_TEST = 'nids_lstm_final_model.h5'
try:
    loaded_model = load_model(MODEL_TO_TEST)
except Exception as e:
    print(f"❌ ไม่พบไฟล์โมเดล {MODEL_TO_TEST} หรือเกิดข้อผิดพลาด: {e}")
    raise


# 3. เริ่มจำลองการตรวจสอบแบบ Real-Time
print("\n--- 💻 เริ่มจำลองการตรวจสอบทราฟฟิกแบบ Real-Time ---")
BATCH_SIZE = 20
num_iterations = 5

for i in range(num_iterations):
    start_index = i * BATCH_SIZE
    end_index = (i + 1) * BATCH_SIZE

    if start_index >= len(X_test_final):
        break

    # ดึงข้อมูล Batch
    current_batch_3d = X_test_final[start_index:end_index]
    current_true_labels = y_test[start_index:end_index]
    current_attack_type_text = attack_type_test.iloc[start_index:end_index].values

    print(f"\n=======================================================")
    print(f"| [รอบที่ {i+1}] ตรวจสอบทราฟฟิกช่วง {start_index}-{end_index-1} |")
    print(f"=======================================================")

    # เรียกฟังก์ชันทำนาย
    results_df, batch_accuracy = real_time_predict(
        current_batch_3d,
        current_true_labels,
        loaded_model,
        threshold=0.9
    )

    # เพิ่มคอลัมน์ชนิดการโจมตี
    results_df.insert(1, 'ชนิดการโจมตี', current_attack_type_text)

    # จัดเรียงคอลัมน์
    display_cols = ['คำทำนาย', 'ชนิดการโจมตี', 'ป้ายกำกับจริง', 'สถานะ', 'ความน่าจะเป็น']
    results_df = results_df[display_cols]

    # แสดงตารางผลลัพธ์
    print(f"\n🔬 **ผลการทำนาย (ความแม่นยำ: {batch_accuracy*100:.2f}%)**")
    display(results_df)
    print("-------------------------------------------------------")

print("\n--- ✅ การจำลองเสร็จสิ้น ---")



--- 💻 เริ่มจำลองการตรวจสอบทราฟฟิกแบบ Real-Time ---

| [รอบที่ 1] ตรวจสอบทราฟฟิกช่วง 0-19 |

🚨 ตรวจพบการบุกรุก 8 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำ: 100.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
1,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.995156e-05
2,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
3,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
4,ปกติ,normal,ปกติ,ถูกต้อง ✅,8.484889e-06
5,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.341433e-08
6,ปกติ,normal,ปกติ,ถูกต้อง ✅,6.999406e-08
7,ปกติ,normal,ปกติ,ถูกต้อง ✅,8.865765e-08
8,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
9,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00


-------------------------------------------------------

| [รอบที่ 2] ตรวจสอบทราฟฟิกช่วง 20-39 |

🚨 ตรวจพบการบุกรุก 8 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำ: 100.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
1,ปกติ,normal,ปกติ,ถูกต้อง ✅,8.340091e-07
2,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.377771e-08
3,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.007457e-08
4,ปกติ,normal,ปกติ,ถูกต้อง ✅,8.605495e-06
5,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
6,ปกติ,normal,ปกติ,ถูกต้อง ✅,4.994261e-08
7,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.738686e-06
8,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.476890e-07
9,ปกติ,normal,ปกติ,ถูกต้อง ✅,6.758761e-10


-------------------------------------------------------

| [รอบที่ 3] ตรวจสอบทราฟฟิกช่วง 40-59 |

🚨 ตรวจพบการบุกรุก 7 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำ: 100.00%)**


...

-------------------------------------------------------

| [รอบที่ 4] ตรวจสอบทราฟฟิกช่วง 60-79 |

🚨 ตรวจพบการบุกรุก 9 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำ: 100.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
1,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.343574e-06
2,ปกติ,normal,ปกติ,ถูกต้อง ✅,7.812848e-06
3,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.211873e-06
4,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
5,ปกติ,normal,ปกติ,ถูกต้อง ✅,6.448401e-05
6,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.016055e-04
7,โจมตี,teardrop,โจมตี,ถูกต้อง ✅,1.000000e+00
8,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.011712e-07
9,โจมตี,nmap,โจมตี,ถูกต้อง ✅,1.000000e+00


-------------------------------------------------------

| [รอบที่ 5] ตรวจสอบทราฟฟิกช่วง 80-99 |

🚨 ตรวจพบการบุกรุก 13 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำ: 100.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.659723e-08
1,โจมตี,ipsweep,โจมตี,ถูกต้อง ✅,9.999995e-01
2,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.029418e-07
3,โจมตี,nmap,โจมตี,ถูกต้อง ✅,1.000000e+00
4,โจมตี,back,โจมตี,ถูกต้อง ✅,9.999956e-01
5,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.102955e-09
6,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.343520e-05
7,ปกติ,normal,ปกติ,ถูกต้อง ✅,9.615692e-04
8,ปกติ,normal,ปกติ,ถูกต้อง ✅,6.622240e-02
9,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.394936e-04


-------------------------------------------------------

--- ✅ การจำลองเสร็จสิ้น ---


In [ ]:
# ***************************************************************
# ** Cell 6: การทดสอบด้วย KDDTest+.txt (External Test Set) **
# ***************************************************************
# ... (ส่วน 1. กำหนดพาธ, โหลดไฟล์, จัดการ Label เหมือนเดิม) ...

# --- B. One-Hot Encoding (ต้องใช้ชุดคอลัมน์เดียวกับชุดฝึก) ---
categorical_cols = ['protocol_type', 'service', 'flag']
test_data_encoded = pd.get_dummies(test_data, columns=categorical_cols, drop_first=True)

# 🔔 ขั้นตอนแก้ไข: ต้องสร้างรายการชื่อคอลัมน์จากชุดฝึก
# เราจะสร้างตัวแปร 'train_cols' จาก X_train_final ใน Cell 2.
# เนื่องจากเราไม่สามารถเข้าถึง Cell 2 ตอนนี้ ผมจะสร้าง 'train_cols' ที่จำเป็นใน Cell 2

# *** ต้องมั่นใจว่าคุณได้รัน Cell 2 ที่มีการสร้างตัวแปร X_scaled แล้ว ***

# ดึงชื่อคอลัมน์ที่ใช้ในการฝึกจาก DataFrame ที่เข้ารหัส (ก่อนทำ X_scaled)
# เนื่องจากเราไม่ได้เก็บ data_encoded (ชุดฝึก) ไว้ใน Cell 2 ผมจะใช้เทคนิคสร้างคอลัมน์ที่คาดหวัง

# *** คุณต้องแก้ไข Cell 2 ให้เก็บชื่อคอลัมน์ชุดฝึกด้วย ***

# สมมติว่าใน Cell 2 เราเพิ่มบรรทัดนี้:
# train_cols = data_encoded.columns
# แต่ถ้าทำไม่ได้ ให้รันโค้ดต่อไปนี้ใน Cell 6 เลย:

# 1. โหลดข้อมูลชุดฝึกอีกครั้ง เพื่อดึงชื่อคอลัมน์ (วิธีที่ง่ายที่สุด)
try:
    train_data_for_cols = pd.read_csv(TRAIN_FILE_PATH, header=None, names=FEATURE_NAMES, sep=',')
    train_data_for_cols = train_data_for_cols.drop(columns=['successful.', 'attack_type'])
    train_data_encoded = pd.get_dummies(train_data_for_cols, columns=categorical_cols, drop_first=True)
    train_cols = train_data_encoded.columns # <-- รายการ 119 คอลัมน์
    print(f"✅ ดึงชื่อคอลัมน์ชุดฝึก: {len(train_cols)} คอลัมน์")
except Exception as e:
    print(f"❌ ERROR: ไม่สามารถโหลดไฟล์ฝึกเพื่อดึงชื่อคอลัมน์ได้: {e}")
    raise


# 2. จัดเรียงคอลัมน์ชุดทดสอบให้ตรงกับชุดฝึก และเติมคอลัมน์ที่ขาดด้วย 0
#    ใช้ reindex() เพื่อจัดเรียงและเติม NaN ในคอลัมน์ที่ไม่มีในชุดทดสอบ
test_data_aligned = test_data_encoded.reindex(columns=train_cols, fill_value=0)

# ตรวจสอบหลังการจัดเรียง
X_test_external = test_data_aligned.values
print(f"⭐ Feature Shape ชุดทดสอบหลังจัดเรียง: {X_test_external.shape[1]}")

if X_test_external.shape[1] != X_train_final.shape[2]:
    print(f"❌ ERROR: จำนวนฟีเจอร์ยังไม่ตรงกัน! คาดหวัง {X_train_final.shape[2]}, ได้ {X_test_external.shape[1]}")
    raise ValueError("Feature alignment failed.")


# --- C. การปรับขนาดข้อมูลฟีเจอร์ (Scaling) ---
# ใช้ global_scaler.transform ได้แล้ว
X_test_scaled_external = global_scaler.transform(X_test_external)


# ... (ส่วน D. Reshape, 2. โหลดโมเดล, 3. จำลอง Real-Time เหมือนเดิม) ...

✅ ดึงชื่อคอลัมน์ชุดฝึก: 119 คอลัมน์
⭐ Feature Shape ชุดทดสอบหลังจัดเรียง: 119


In [ ]:
# ***************************************************************
# ** Cell 6: การทดสอบด้วย KDDTest+.txt (External Test Set) **
# *** แก้ไขปัญหา ValueError: Feature Mismatch (113 vs 119) แล้ว ***
# ***************************************************************
print("--- 🔬 เริ่มกระบวนการทดสอบด้วย KDDTest+.txt ---")

import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from IPython.display import display

# 1. กำหนดพาธไฟล์
TEST_FILE_PATH = '/content/drive/MyDrive/dataset/KDDTest+.txt'
TRAIN_FILE_PATH_FOR_COLS = '/content/drive/MyDrive/dataset/KDDTrain+.txt'
MODEL_TO_TEST = 'nids_lstm_final_model.h5'

FEATURE_NAMES = [
    'duration', 'protocol_type', 'service', 'flag', 'src_bytes', 'dst_bytes', 'land',
    'wrong_fragment', 'urgent', 'hot', 'num_failed_logins', 'logged_in', 'num_compromised',
    'root_shell', 'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
    'num_access_files', 'num_outbound_cmds', 'is_host_login', 'is_guest_login',
    'count', 'srv_count', 'serror_rate', 'srv_serror_rate', 'rerror_rate',
    'srv_rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_diff_host_rate',
    'dst_host_count', 'dst_host_srv_count', 'dst_host_same_srv_rate',
    'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
    'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
    'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
    'dst_host_srv_rerror_rate', 'attack_type', 'successful.'
]

# -------------------------------------------------------------
# โหลดชุดข้อมูลทดสอบ
# -------------------------------------------------------------
try:
    test_data = pd.read_csv(TEST_FILE_PATH, header=None, names=FEATURE_NAMES, sep=',')
    test_data = test_data.drop(columns=['successful.'])
    print(f"✅ โหลดไฟล์: {TEST_FILE_PATH} สำเร็จแล้ว!")
except FileNotFoundError:
    print(f"❌ ERROR: ไม่พบไฟล์ทดสอบที่พาธ {TEST_FILE_PATH}")
    raise

# --- A. การจัดการ Label และ Attack Type ---
attack_type_test_external = test_data['attack_type'].copy()
test_data['attack_type'] = test_data['attack_type'].apply(lambda x: 0 if x == 'normal' else 1)
y_test_external = test_data['attack_type'].values
test_data = test_data.drop(columns=['attack_type'])

# --- B. One-Hot Encoding และจัดการ Feature Mismatch ---
categorical_cols = ['protocol_type', 'service', 'flag']
test_data_encoded = pd.get_dummies(test_data, columns=categorical_cols, drop_first=True)

try:
    train_data_for_cols = pd.read_csv(TRAIN_FILE_PATH_FOR_COLS, header=None, names=FEATURE_NAMES, sep=',')
    train_data_for_cols = train_data_for_cols.drop(columns=['successful.', 'attack_type'])
    train_data_encoded = pd.get_dummies(train_data_for_cols, columns=categorical_cols, drop_first=True)
    train_cols = train_data_encoded.columns
    print(f"✅ ดึงชื่อคอลัมน์ชุดฝึก: {len(train_cols)} คอลัมน์")
except Exception as e:
    print(f"❌ ERROR: ไม่สามารถโหลดไฟล์ฝึกเพื่อดึงชื่อคอลัมน์ได้: {e}")
    raise

# จัดเรียงคอลัมน์ชุดทดสอบให้ตรงกับชุดฝึก
test_data_aligned = test_data_encoded.reindex(columns=train_cols, fill_value=0)
X_test_external = test_data_aligned.values
print(f"⭐ Feature Shape ชุดทดสอบหลังจัดเรียง: {X_test_external.shape[1]}")

# --- C. Scaling ---
X_test_scaled_external = global_scaler.transform(X_test_external)

# --- D. Reshape สำหรับ LSTM ---
timesteps = 1
features = X_test_scaled_external.shape[1]
X_test_final_external = X_test_scaled_external.reshape(X_test_scaled_external.shape[0], timesteps, features)
print(f"✅ Input Shape สำหรับ LSTM (X_test_final_external): {X_test_final_external.shape}")

# -------------------------------------------------------------
# 2. โหลดโมเดลและประเมินผล
# -------------------------------------------------------------
try:
    loaded_model = load_model(MODEL_TO_TEST)
except Exception as e:
    print(f"❌ ไม่พบไฟล์โมเดล {MODEL_TO_TEST} หรือเกิดข้อผิดพลาดในการโหลด: {e}")
    raise

loss_ext, accuracy_ext = loaded_model.evaluate(X_test_final_external, y_test_external, verbose=0)
print(f"\n=======================================================")
print(f"⭐ **ผลลัพธ์ความแม่นยำรวม (KDDTest+): {accuracy_ext*100:.2f}%**")
print(f"=======================================================")

# -------------------------------------------------------------
# 3. จำลอง Real-Time (ใช้โค้ดจาก Cell 5)
# -------------------------------------------------------------
print("\n--- 💻 เริ่มจำลองการตรวจสอบทราฟฟิกแบบ Real-Time (KDDTest+) ---")
BATCH_SIZE = 20
num_iterations = 5

for i in range(num_iterations):
    start_index = i * BATCH_SIZE
    end_index = (i + 1) * BATCH_SIZE

    if start_index >= len(X_test_final_external):
        break

    current_batch_3d = X_test_final_external[start_index:end_index]
    current_true_labels = y_test_external[start_index:end_index]
    current_attack_type_text = attack_type_test_external.iloc[start_index:end_index].values

    print(f"\n=======================================================")
    print(f"| [รอบที่ {i+1}] ตรวจสอบทราฟฟิกช่วง {start_index}-{end_index-1} |")
    print(f"=======================================================")

    # ใช้ฟังก์ชันจาก Cell 5 (ไม่มีสี)
    results_df, batch_accuracy = real_time_predict(
        current_batch_3d,
        current_true_labels,
        loaded_model,
        threshold=0.9
    )

    # เพิ่มคอลัมน์ชนิดการโจมตี
    results_df.insert(1, 'ชนิดการโจมตี', current_attack_type_text)

    # จัดเรียงคอลัมน์และแสดงผล
    display_cols = ['คำทำนาย', 'ชนิดการโจมตี', 'ป้ายกำกับจริง', 'สถานะ', 'ความน่าจะเป็น']
    results_df = results_df[display_cols]

    print(f"\n🔬 **ผลการทำนาย (ความแม่นยำในชุดนี้: {batch_accuracy*100:.2f}%)**")
    display(results_df)
    print("-------------------------------------------------------")

print("\n--- ✅ การทดสอบด้วย KDDTest+.txt เสร็จสิ้น ---")


--- 🔬 เริ่มกระบวนการทดสอบด้วย KDDTest+.txt ---
✅ โหลดไฟล์: /content/drive/MyDrive/dataset/KDDTest+.txt สำเร็จแล้ว!
✅ ดึงชื่อคอลัมน์ชุดฝึก: 119 คอลัมน์
⭐ Feature Shape ชุดทดสอบหลังจัดเรียง: 119
✅ Input Shape สำหรับ LSTM (X_test_final_external): (22544, 1, 119)



⭐ **ผลลัพธ์ความแม่นยำรวม (KDDTest+): 79.71%**

--- 💻 เริ่มจำลองการตรวจสอบทราฟฟิกแบบ Real-Time (KDDTest+) ---

| [รอบที่ 1] ตรวจสอบทราฟฟิกช่วง 0-19 |

🚨 ตรวจพบการบุกรุก 6 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำในชุดนี้: 75.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
1,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
2,ปกติ,normal,ปกติ,ถูกต้อง ✅,5.839378e-06
3,โจมตี,saint,โจมตี,ถูกต้อง ✅,1.000000e+00
4,ปกติ,mscan,โจมตี,ผิดพลาด ❌,6.320020e-05
5,ปกติ,normal,ปกติ,ถูกต้อง ✅,9.468819e-07
6,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.852749e-03
7,ปกติ,guess_passwd,โจมตี,ผิดพลาด ❌,1.663659e-06
8,ปกติ,normal,ปกติ,ถูกต้อง ✅,9.713062e-08
9,ปกติ,guess_passwd,โจมตี,ผิดพลาด ❌,2.209543e-07


-------------------------------------------------------

| [รอบที่ 2] ตรวจสอบทราฟฟิกช่วง 20-39 |

🚨 ตรวจพบการบุกรุก 9 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำในชุดนี้: 90.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
1,โจมตี,mscan,โจมตี,ถูกต้อง ✅,9.498580e-01
2,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.414282e-05
3,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.030225e-08
4,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
5,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
6,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.690955e-07
7,ปกติ,normal,ปกติ,ถูกต้อง ✅,7.229603e-05
8,โจมตี,smurf,โจมตี,ถูกต้อง ✅,1.000000e+00
9,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.465686e-01


...

-------------------------------------------------------

| [รอบที่ 4] ตรวจสอบทราฟฟิกช่วง 60-79 |

🚨 ตรวจพบการบุกรุก 6 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำในชุดนี้: 85.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,ปกติ,guess_passwd,โจมตี,ผิดพลาด ❌,3.852617e-08
1,ปกติ,warezmaster,โจมตี,ผิดพลาด ❌,1.967933e-01
2,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.287381e-04
3,ปกติ,normal,ปกติ,ถูกต้อง ✅,3.117335e-06
4,โจมตี,satan,โจมตี,ถูกต้อง ✅,1.000000e+00
5,ปกติ,normal,ปกติ,ถูกต้อง ✅,6.455985e-06
6,โจมตี,mscan,โจมตี,ถูกต้อง ✅,1.000000e+00
7,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
8,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
9,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.950320e-05


-------------------------------------------------------

| [รอบที่ 5] ตรวจสอบทราฟฟิกช่วง 80-99 |

🚨 ตรวจพบการบุกรุก 7 แถว (Threshold = 0.9)

🔬 **ผลการทำนาย (ความแม่นยำในชุดนี้: 90.00%)**


,คำทำนาย,ชนิดการโจมตี,ป้ายกำกับจริง,สถานะ,ความน่าจะเป็น
0,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
1,โจมตี,neptune,โจมตี,ถูกต้อง ✅,1.000000e+00
2,โจมตี,pod,โจมตี,ถูกต้อง ✅,9.999992e-01
3,ปกติ,normal,ปกติ,ถูกต้อง ✅,4.268054e-08
4,ปกติ,normal,ปกติ,ถูกต้อง ✅,9.496849e-06
5,ปกติ,normal,ปกติ,ถูกต้อง ✅,2.516432e-08
6,โจมตี,neptune,โจมตี,ถูกต้อง ✅,9.975866e-01
7,โจมตี,mscan,โจมตี,ถูกต้อง ✅,8.460053e-01
8,ปกติ,normal,ปกติ,ถูกต้อง ✅,1.116538e-04
9,ปกติ,normal,ปกติ,ถูกต้อง ✅,6.445358e-04


-------------------------------------------------------

--- ✅ การทดสอบด้วย KDDTest+.txt เสร็จสิ้น ---
